In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 47.3 MB 2.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 9.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595749 sha256=d45a9224b5b9b1ceb558e332e0861be26afcdeaee3a81dca0e3405f7b124e9fa
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 4.0 MB 9.6 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 880 kB 63.3 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=00a7ab65e79e19d1aeaf396ad9d04dc0045b38

In [ ]:
!pip install simpletransformers


     |████████████████████████████████| 249 kB 7.1 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 10.1 MB 66.3 MB/s 
     |████████████████████████████████| 1.8 MB 52.9 MB/s 
     |████████████████████████████████| 325 kB 77.5 MB/s 
     |████████████████████████████████| 181 kB 76.1 MB/s 
     |████████████████████████████████| 144 kB 74.2 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 62.3 MB/s 
     |████████████████████████████████| 212 kB 77.7 MB/s 
     |████████████████████████████████| 136 kB 75.2 MB/s 
     |████████████████████████████████| 127 kB 79.0 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 60.6 MB/s 
     |████████████████████████████████| 271 kB 48.2 MB/s 
     |████████████████████████████████| 164 kB 55.7 MB/s 
     |████████████████████████████████| 4.3 MB 25.2 MB/s 
     |██████████████

In [ ]:
import transformers

In [ ]:
import os 
import random 
import time 
import torch 

import pandas as pd 
import numpy as np 
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, BertConfig 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences

import datetime

import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/PIG/한국어_단발성_대화_데이터셋.xlsx')
df.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  38594 non-null  object
 1   Emotion   38594 non-null  object
dtypes: object(2)
memory usage: 603.2+ KB


In [ ]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,Sentence,Emotion
0,나토 어찌냐.... 이슬람 국가 세력 더 넓어지겠네...,중립
1,저렴한 가격에 쉬운 재료로 어쩌면 그렇게 쉽고 맛있게 할수잇는지감사합니다.,행복
2,막가 얍샾 두놈이서 놀고 자빠졌구나 천벌받을 놈들이다.,분노
3,우리같으면 무조건 찬성하고 돈받고 보자 했을텐데..,중립
4,개봉일에 봤는데 앞으로 1번 이상 다시 보러 갈거에요 !!!,행복


In [ ]:
print(df_shuffled.shape)

(38594, 2)


In [ ]:
df_shuffled.loc[(df_shuffled['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
df_shuffled.loc[(df_shuffled['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
df_shuffled.loc[(df_shuffled['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
df_shuffled.loc[(df_shuffled['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
df_shuffled.loc[(df_shuffled['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
df_shuffled.loc[(df_shuffled['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
df_shuffled.loc[(df_shuffled['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
dataset_train, dataset_test = train_test_split(df_shuffled, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

dataset_train.head()

28945
9649


,Sentence,Emotion
29186,그런데 저런 배아픈거보다신물이 너무 올라와서 힘들어요ㅠ,3
17237,2013년에는 백상예술대상 타셨지,4
22295,엄머낫 왠일이래 정말 부부사이는 부부만 아나부다~,1
8161,필리핀 에서도 박보검 인기가 ㅎㄷㄷ,1
11873,비가 오면 좋은 징조다,5


In [ ]:
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification
from torch.utils.data import DataLoader, Dataset


In [ ]:
# XLM-RoBERTa 토크나이저를 불러옵니다: https://huggingface.co/xlm-roberta-large
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'xlm-roberta-large'
# 만약 colab pro가 아니면 MODEL_TYPE = 'xlm-roberta-base'를 사용하세요
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification
from torch.utils.data import DataLoader, Dataset


In [ ]:
tokenizer.vocab_size

250002

In [ ]:
list(tokenizer.get_vocab())[:10]

['<s>', '<pad>', '</s>', '<unk>', ',', '.', '▁', 's', '▁de', '-']

In [ ]:
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [ ]:
row = df_shuffled.iloc[0, 0:2].values
text = row[0]
y = row[1]
print(row)
print(text)
print(y)

['나토 어찌냐.... 이슬람 국가 세력 더 넓어지겠네...' 4]
나토 어찌냐.... 이슬람 국가 세력 더 넓어지겠네...
4


In [ ]:
class CompDataset(Dataset):

    def __init__(self, csv_file):
        self.dataset = csv_file
        self.tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

        print(self.dataset.describe())

    def __getitem__(self, idx):

        row = self.dataset.iloc[idx, 0:2].values
        text = row[0]
        y = row[1]

        encoded_dict = tokenizer.encode_plus(
                    text,      
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        input_ids = encoded_dict['input_ids'][0]
        attention_mask = encoded_dict['attention_mask'][0]

        return input_ids, attention_mask, y

    def __len__(self):
        return len(self.dataset)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, csv_file):
        self.dataset = csv_file
        self.tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
    def __getitem__(self, idx):

        row = self.dataset.iloc[idx, 0:1].values
        text = row[0]



        encoded_dict = tokenizer.encode_plus(
                    text,     
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        input_ids = encoded_dict['input_ids'][0]
        attention_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
     

        return input_ids, attention_mask


    def __len__(self):
        return len(self.dataset)

In [ ]:
L_RATE = 1e-5 
MAX_LEN = 32 

BATCH_SIZE = 32 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

NUM_CORES

2

In [ ]:
from transformers import XLMRobertaForSequenceClassification


import gc
gc.collect()
torch.cuda.empty_cache()
model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 7, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

In [ ]:
train_data = CompDataset(dataset_train)
test_data = CompDataset(dataset_test)

# batch_size 만큼 데이터 분할
train_loader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=0)


test_loader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=0)



print(len(train_loader))

print(len(test_loader))

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

            Sentence  Emotion
count          28945    28945
unique         28901        7
top      남자친구가 떠날까봐요        5
freq               3     4522
       Sentence  Emotion
count      9649     9649
unique     9641        7
top       감사합니다        5
freq          3     1515
905
302


In [ ]:
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:

gc.collect()
torch.cuda.empty_cache()

from tqdm.notebook import tqdm
from torch.nn import functional as F

# train
losses = []
accuracies = []
total_loss = 0.0
correct = 0
total = 0
epochs = 5
for i in range(epochs):

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

    losses.append(total_loss)
    accuracies.append(correct.float() / total)
    print("Train Loss:", total_loss / total, "Accuracy:", correct.float() / total)

  0%|          | 0/905 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Train Loss: 0.04346528859681304 Accuracy: tensor(0.4526, device='cuda:0')


  0%|          | 0/905 [00:00<?, ?it/s]

Train Loss: 0.03925950101958509 Accuracy: tensor(0.5132, device='cuda:0')


...

Train Loss: 0.036730560086349645 Accuracy: tensor(0.5473, device='cuda:0')


  0%|          | 0/905 [00:00<?, ?it/s]

Train Loss: 0.03464446837990822 Accuracy: tensor(0.5743, device='cuda:0')


  0%|          | 0/905 [00:00<?, ?it/s]

Train Loss: 0.03274067255906602 Accuracy: tensor(0.5989, device='cuda:0')


In [ ]:
# validation
model.eval()

pred = []
correct = 0
total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  pred.append(predicted)
  correct += (predicted == y_batch).sum()
  total += len(y_batch)

print("val accuracy:", correct.float() / total)

  0%|          | 0/302 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


val accuracy: tensor(0.5707, device='cuda:0')


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence]
    dataset_another = pd.DataFrame(data)

    another_test = TestDataset(dataset_another)
    test_dataloader = DataLoader(another_test, batch_size=BATCH_SIZE, num_workers=5)
    
    model.eval()

    for input_ids_batch, attention_masks_batch in tqdm(test_dataloader):
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        _, predicted = torch.max(y_pred, 1)
        pred.extend(predicted.tolist())


        test_eval=[]
        for i in y_pred:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 중간고사를 봤다. 시험을 망쳤다. 그래도 찍은거 하나는 맞췄다.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘은 영화를 봤다. 그런데 개노잼이었다.


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 코로나는 대체 언제 없어지는거야?


  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


>> 입력하신 내용에서 놀람이 느껴집니다.


